In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from matplotlib.pyplot import figure
from IPython.display import clear_output

In [ ]:
# Import dataset
raw_data = pd.read_csv("../../datasets/raw/fire_nrt_V1_95405.csv") 
raw_data.head()

In [ ]:
def select_timeframe(dataframe, start, end):
    """
    select a start and an end date (int) of the 92 days of available data
    """
    
    assert end - start > 0    
    date_range = dataframe.acq_date.unique()[start : end]
    
    return dataframe.loc[dataframe['acq_date'].isin(date_range)].reset_index()

In [ ]:
def normalize_coordinates(input_data, x_scale, y_scale, max_lat, min_lat, max_lon, min_lon):
    lat = input_data.latitude
    lon = input_data.longitude
    
    # Normalize coordinates
    df = pd.DataFrame()
    df['lat'] = (lat - min_lat) / (max_lat - min_lat)
    df['lon'] = (lon - min_lon) / (max_lon - min_lon)

#     assert df.lat.min() == 0.0 and df.lat.max() == 1.0
#     assert df.lon.min() == 0.0 and df.lon.max() == 1.0

    df['lon'] = df.lon * (x_scale - 1)
    df['lat'] = df.lat * (y_scale - 1)

#     assert df.lon.max() == float(x_scale - 1) and df.lat.max() == float(y_scale - 1)

    # Round
    df['lon'] = round(df.lon)
    df['lat'] = round(df.lat)
    
#     print('Extrema latitude: ', min_lat, max_lat, '\nExtrema longitude:', min_lon, max_lon)
#     print('Ratio: ', abs(min_lat - max_lat) / abs(min_lat - max_lat), abs((min_lon - max_lon) / abs(min_lat - max_lat)))
#     print('Length: ', len(data))
    
    return df

In [ ]:
def construct_heatmap(dataframe, in_x, in_y, out_x, out_y, scale_on=True):
    im = np.zeros((in_y, in_x))
    heat_range = len(dataframe)
    
    for i in range(heat_range):
        if im[int(dataframe.lat[i]), int(dataframe.lon[i])] == 0:
            im[int(dataframe.lat[i]), int(dataframe.lon[i])] = 1

        else:
            if scale_on is True:
                im[int(dataframe.lat[i]), int(dataframe.lon[i])] += 0

    # Add ocean buffer
    bottom_ocean = np.zeros((int((in_x - in_y) / 2), in_x))
    height = (int((in_x - in_y) / 2) + in_y)
    side_ocean = np.zeros((height, int(height * 50/1000)))
    
    # bottom top
    im1 = np.concatenate((bottom_ocean, im), 0)

    # left right
    im1 = np.concatenate((side_ocean, im1, side_ocean), 1)
    
    im1 = cv2.resize(im1, (out_x, out_y), interpolation=cv2.INTER_NEAREST)
    return im1

In [ ]:
def plot_data(data, mask_image, date):
    shape = np.shape(data)    
    mask_image = cv2.resize(mask_image, (shape[1], shape[0]), interpolation=cv2.INTER_NEAREST)
    
    assert np.shape(data) == np.shape(mask_image)

    figure(num=None, figsize=(16, 16))
    im = np.ma.masked_where(mask_image == 0, data)
    
    # Plot
    cmap = plt.cm.OrRd
    plt.imshow(im, cmap='plasma', interpolation='nearest', origin='lower', vmin=0, vmax=3)
    plt.colorbar()
    plt.title(date)
    plt.show()
    
    clear_output(wait=True)

In [ ]:
mask = np.load('australia-vegetation-heatmap.npy')

x_in = 220
y_in = 180
x_out = 1100
y_out = 900

def animate(raw_data, mask, x_in, y_in, x_out, y_out, iterations):
    to_plot = None
    dates = raw_data.acq_date.unique()
    
    # Set extrema
    lat = raw_data.latitude
    lon = raw_data.longitude

    max_lat = max(lat)
    min_lat = min(lat)
    max_lon = max(lon)
    min_lon = min(lon)
    
    for i in range(iterations):
        
        step1 = select_timeframe(raw_data, i, i+1)
        step2 = normalize_coordinates(step1, x_in, y_in, max_lat, min_lat, max_lon, min_lon)
        step3 = construct_heatmap(step2, x_in, y_in, x_out, y_out, False)
        
        if to_plot is None:
            to_plot = step3
        else:
            to_plot *= 0.90
            to_plot += step3
        
        plot_data(to_plot, mask, dates[i])
        
animate(raw_data, mask, x_in, y_in, x_out, y_out, 93)

In [ ]:
# Interactive visualization
import ipywidgets as widgets

x_in = 220
y_in = 180
x_out = 1100
y_out = 900

lat = raw_data.latitude
lon = raw_data.longitude

max_lat = max(lat)
min_lat = min(lat)
max_lon = max(lon)
min_lon = min(lon)

@widgets.interact
def f(i=5):
    to_plot = None
    dates = raw_data.acq_date.unique()
    
    step1 = select_timeframe(raw_data, i, i+1)
    step2 = normalize_coordinates(step1, x_in, y_in, max_lat, min_lat, max_lon, min_lon)
    step3 = construct_heatmap(step2, x_in, y_in, x_out, y_out, False)

    to_plot = step3

    plot_data(to_plot, mask, dates[i])


In [ ]:
import ipywidgets as widgets

play = widgets.Play(
#     interval=10,
    value=50,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)

slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.HBox([play, slider])


